#  **COMM475--Investment Policies:Implementation of Chen, Roll, and Ross (1986) Estimation Methodology**

Instructor: Lorenzo Garlappi © 2024

TA: Tianping Wu




In this notebook we replicate the procedure used by Chen, Roll, and Ross~(Journal of Business, 1986) to estimate factor risk premia $\lambda$'s.

We use 10 industry portfolios from French library.

Let's first import packages:

In [17]:
import numpy as np
import statsmodels.api as sm #package for the regression
from datetime import datetime
import pandas as pd

<span style="color: red;"> Tianping, the code below is not good. Suppose we want to use 20 portfolio instead of 10. Then everything is messed up. </span> 

<span style="color: red;">I suggest the following:</span> 

<span style="color: red;">1. The data should be in two files: 1 files for the N portfolios and one file for the 5 shocks I </span> 

<span style="color: red;">2. Then merge the portfolios with the shocks to create a dataframe that will be used for regressions</span> 


<span style="color: red;">What are the factors? WHat does I_1, I_2 refer to? This is important for interpretation! How are we making sense of whether the estimation is reasonable of not?</span> 


We then read the data:

In [18]:
#read the data
data_df = pd.read_csv('10_Industry_Portfolios.csv')

# Let's take a peek at how the data looks like
data_df.head()

,YYYYMM,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,I1,I2,I3,I4,I5
0,192607,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13,0.041926,0.668327,0.006839,0.787482,0.062262
1,192608,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35,0.984277,0.668150,0.272623,0.678606,0.373163
2,192609,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29,0.807803,0.996071,0.347169,0.820643,0.779482
3,192610,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84,0.879134,0.984930,0.705271,0.259661,0.300433
4,192611,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11,0.812806,0.371622,0.817885,0.135713,0.758548


In [21]:
# Report the mean of I1 to I5
data_df[['I1','I2','I3','I4','I5']].mean()


I1    0.503418
I2    0.495881
I3    0.499049
I4    0.487144
I5    0.502601
dtype: float64

<span style="color: red;"> Tianping, the Factor shocks $I_i$ should have a mean of approximately zero. Please see above. It does not look like these are factor shocks!</span> 

We pre process the data:

In [24]:
# Convert the 'YYYYMM' column to datetime format
data_df['Date'] = pd.to_datetime(data_df['YYYYMM'], format='%Y%m')

# Set the 'Date' column as the index
data_df.set_index('Date', inplace=True)

# remove the 'YYYYMM' column
data_df.drop('YYYYMM', axis=1, inplace=True)

# Let's take a peek at what we have accomplished...can you tell? Hint: compare with the previous table above
data_df.head()

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,I1,I2,I3,I4,I5
Date,,,,,,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13,0.041926,0.668327,0.006839,0.787482,0.062262
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35,0.984277,0.668150,0.272623,0.678606,0.373163
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29,0.807803,0.996071,0.347169,0.820643,0.779482
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84,0.879134,0.984930,0.705271,0.259661,0.300433
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11,0.812806,0.371622,0.817885,0.135713,0.758548


In [28]:
# THIS PART OF THE CODE IS NOT GOOD AS IT IS DATA SPECIFIC. WE NEED TO MAKE IT MORE GENERAL. SEE COMMENTS ABOVE

# select the headings labels of the dtaframe
headings = data_df.columns.values.tolist()
# construct a string with headings from the second to the 11th
headings_industries = headings[1:10]
# construct a string with headings from the 12th to the end
headings_factors = headings[10:]

print(headings_industries)

print(headings_factors)



['Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth ', 'Utils', 'Other']
['I1', 'I2', 'I3', 'I4', 'I5']


## CRR procedure to estimate $\beta$ and $\lambda$

### Step A:Choose the start and end month

In [29]:
# Define the start month and end month for the analysis
start_month = pd.to_datetime('1970-01-01')
end_month = pd.to_datetime('1984-12-01')

### Step B: Use previous M months to estimate factor sensitivities, $\beta_{j p}$, $p=1,\ldots, N$:

$$
r_{p}=\alpha_{p}+ \beta_{1 p}I_{1}+\beta_{2 p}I_{2} + \beta_{3 p}I_{3}+ \beta_{4 p}I_{4}+ \beta_{5 p}I_{5} + e_{s}
$$

By 10 regressions (for N different portfolios, $p$ ), each with M observations. This gives estimates of $\beta_{j p}$ for $j=1,\ldots, 5$ and $p=1,\ldots,N$ for each month

We first define the function to estimate beta:

<span style="color: red;"> Tianping, See comment above. If we start with two datasets the following function will be much cleaner. </span>

<span style="color: red;"> Please see examples of Fama-MacBeth implementation (Python) here:
https://www.tidy-finance.org/python/fama-macbeth-regressions.html  </span>


<span style="color: red;"> Please see examples of Fama-MacBeth implementation (R) here:
https://www.tidy-finance.org/r/fama-macbeth-regressions.html </span>


In [31]:
# This function estimates betas by regressing returns on shocks

def estimate_beta(returns, shocks):
    # Ensure that the returns and shocks have the same index
    common_index = returns.index.intersection(shocks.index)
    returns = returns.loc[common_index] # portfolio returns
    shocks = shocks.loc[common_index]   # economic shocks

    # Add a constant to the model (for the intercept)
    X = sm.add_constant(shocks)

    # Perform the regression of portfolio returns on the economic shocks
    model = sm.OLS(returns, X).fit()

    # Return the estimated coefficients
    return model.params

<span style="color: red;"> Tianping, This is not good. Why are you hard coding in the labels of shocks and industry portfolios? What if we change the data? We need to rewrite the code!!! Bad programming.... </span> 


We define five economic shocks:

In [32]:
# five economic shocks
shocks = ['I1', 'I2', 'I3', 'I4', 'I5']

We define ten industry portfolios:

In [33]:
#ten industry portfolios
portfolios = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth ', 'Utils', 'Other']

We then generate the beta for each portfolio in each month:

In [34]:
# Placeholder for storing the beta estimates for each month
monthly_beta_estimates = {}

# 12 months in a year
num_of_month = 12

def calculate_month_diff(timestamp1, timestamp2):
    months_diff = (timestamp2.year - timestamp1.year) * num_of_month + timestamp2.month - timestamp1.month
    return months_diff

num_month = calculate_month_diff(start_month, end_month) + 1     
#number of the month from start month to end month

# Loop through each month from January 1970 to December 1984
for i in range(num_month):
    # Determine the 60-month period that precedes the current month
    period_start = start_month - pd.DateOffset(months = 60 - i)
    period_end = period_start + pd.DateOffset(months = 60)
    
    # Filter the data for this 60-month period
    period_data = data_df[(data_df.index >= period_start) & (data_df.index < period_end)]
    
    # give the value of economic shocks in the period
    shocks_period = period_data[shocks]
    
    # give the value of portfolio returns in the period
    portfolios_period = period_data[portfolios]
    
    # Estimate the betas for each portfolio using the filtered data
    betas = ['beta' + str(i + 1) for i in range(len(shocks))]
    df = estimate_beta(portfolios_period, shocks_period).T
    df.columns = ['const'] + betas
    df = df.drop(['const'], axis = 1)
    
    # Import the return of each portfolio
    df_return = data_df.loc[period_end] 
    df['return'] = df_return[portfolios].values
    df.index = df_return[portfolios].index
    
    # Store the results for the current month
    monthly_beta_estimates[period_end] = df


In [35]:
print(start_month)
print(end_month)
print(num_month)

check_month = pd.to_datetime('1975-01-01')

monthly_beta_estimates[check_month]




1970-01-01 00:00:00
1984-12-01 00:00:00
180


,beta1,beta2,beta3,beta4,beta5,return
NoDur,3.137777,0.630785,2.516569,0.018421,-1.581685,18.71
Durbl,4.372689,1.148103,2.920121,-0.542038,-3.304216,21.50
Manuf,4.498989,0.485731,2.445305,0.866069,-1.667379,14.51
Enrgy,5.066745,0.494832,2.191484,3.459871,1.678021,6.77
HiTec,5.415134,-1.112840,1.191687,3.322604,-3.813928,15.30
Telcm,1.878771,0.914853,2.667278,0.474416,2.271305,11.09
Shops,3.154919,0.627481,4.619016,-1.044179,-1.702778,25.92
Hlth,2.104528,-0.187913,0.405010,1.288258,-0.852895,-0.74
Utils,1.673071,2.294631,3.983118,1.550374,1.010982,18.80
Other,3.485292,1.162302,4.362794,1.810095,-0.941320,17.89


In [11]:
## You are free to search betas for each month

year = 1978
month = 7

year_start = start_month.year #1970
month_start = start_month.month #1

#the rank of the month from the start month
key_num = (year - year_start)* num_of_month + month - month_start

list(monthly_beta_estimates.values())[key_num]

,beta1,beta2,beta3,beta4,beta5,return
NoDur,0.665286,2.657170,1.149101,-1.069498,2.314799,3.53
Durbl,0.847814,1.208884,1.961945,-1.948213,1.623510,4.71
Manuf,0.763417,0.509003,1.292948,-1.911371,2.010197,7.84
Enrgy,3.015106,0.036892,-0.397485,0.334006,3.020416,3.34
HiTec,0.044239,0.425846,1.020515,-2.648755,0.024859,8.58
Telcm,0.605490,3.314086,-1.304465,-0.430889,2.503687,1.42
Shops,1.544209,3.494777,4.015422,-2.447961,2.481368,5.14
Hlth,1.640742,-0.099942,-1.010888,-2.565533,-2.132916,8.86
Utils,1.148541,3.090397,0.645027,0.952702,3.994189,2.83
Other,1.243161,2.318321,0.951220,-0.972309,3.186253,6.61


### Step C: For each month, do a cross-sectional regression with N observations:

$$
E[r_{p}]=\mu+ \hat{\beta}_{1 p}\lambda_{1}+ \hat{\beta}_{2 p}\lambda_{2}+\hat{\beta}_{3 p}\lambda_{3} + \hat{\beta}_{4 p}\lambda_{4}+ \hat{\beta}_{5 p}\lambda_{5}+e_{s},~~~p=1,\ldots,N
$$

Gives estimates of market prices of risk for each month: $\lambda_{j}, j=1,5$.

We define the function to estmate the price of risk lambda:

In [36]:
# Estimates the market prices of risk (lambda coefficients) using a cross-sectional regression.
def estimate_lambda(betas, returns):
    
#avg_beta: average beta coefficients for portfolios at month t.
#avg_returns: average returns of portfolios at month t.
    

    # Add a constant to the model
    X = sm.add_constant(betas)

    # Perform the cross-sectional regression
    model = sm.OLS(returns, X).fit()

    # Return the estimated lambda coefficients
    return model.params

estimate the lambda for each period:

In [37]:
# create a dataframe for storing lambda
df_lambda = pd.DataFrame(np.ones((num_month, len(betas) + 1)))
df_lambda.columns = ['const'] + ['lambda' + str(i+1) for i in range(len(betas))]
df_lambda.index = [start_month + pd.DateOffset(months = i) for i in range(num_month)]
months = list(monthly_beta_estimates.keys())

# estimate lambda for each regression
for i in range(num_month):
    df_reg = monthly_beta_estimates[months[i]]
    params = estimate_lambda(df_reg[betas], df_reg['return'])
    df_lambda.iloc[i,] = params
df_lambda

,const,lambda1,lambda2,lambda3,lambda4,lambda5
1970-01-01,-1.198546,0.211796,0.575021,2.180185,1.587768,-1.239200
1970-02-01,8.295244,0.239491,1.405527,1.705438,-1.350824,-0.781968
1970-03-01,2.868445,1.271208,2.108749,-0.429520,1.584857,-1.610826
1970-04-01,-6.888381,0.345788,0.315584,-0.159524,1.199253,0.943100
1970-05-01,-2.440920,-0.637066,1.340030,-0.248352,-0.268171,1.935764
...,...,...,...,...,...,...
1984-08-01,5.020053,1.129373,-1.027296,-0.631714,-1.002149,-0.834809
1984-09-01,3.668726,-0.086570,1.863161,-0.110453,0.322354,0.197663
1984-10-01,5.333655,0.981506,0.111298,-3.697479,-1.790456,-1.060600
1984-11-01,4.320006,0.085958,1.010917,-0.782054,1.291773,-0.063131


In [38]:
# Smmarize the lambda coefficients
df_lambda.describe()

,const,lambda1,lambda2,lambda3,lambda4,lambda5
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,0.900128,-0.036017,0.033052,-0.172368,-0.009074,0.081887
std,6.031593,1.413889,1.817341,1.506931,1.468694,1.544072
min,-15.377357,-4.772346,-8.113575,-4.557367,-4.368100,-6.235447
25%,-2.319447,-0.725687,-0.769198,-1.005546,-0.942636,-0.995273
50%,0.185437,0.026949,0.087197,-0.203663,-0.029862,0.025227
75%,4.239868,0.736859,1.029629,0.503410,1.134254,0.950243
max,28.662395,4.125761,4.616374,5.142610,3.363218,5.823428


In [15]:


#  compute the time series average of the lambda coefficients
df_lambda.mean()

# compute the standard error of the lambda coefficients across time
df_lambda.std()


const      6.031593
lambda1    1.413889
lambda2    1.817341
lambda3    1.506931
lambda4    1.468694
lambda5    1.544072
dtype: float64

You are free to get the average lambda in each year:

In [16]:
df_lambda.loc['1979'].mean()

const     -0.610225
lambda1    0.806715
lambda2   -0.650372
lambda3   -0.198994
lambda4   -0.707285
lambda5    0.549215
dtype: float64

<span style="color: red;">THE RESULTS ABOVE CANNOT BE RIGHT. YOU ESTIMATE MOST RISK PREMIA TO BE NEGATIVE. THINK ABOUT WHAT THIS MEANS!!!</span> 
